<h1 id="tocheading">Spring 2018 NLP Class Project: Neural Machine Translation</h1>
<div id="toc"></div>

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import pandas as pd
import spacy
import pdb
import os
from underthesea import word_tokenize
import jieba
import numpy as np

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# ! pip install spacy && python -m spacy download en

## Part 0: Project Overview

The goal of this project is to build a neural machine translation system and experience how recent advances have made their way. Each team will build the following sequence of neural translation systems for two language pairs, __Vietnamese (Vi)→English (En)__ and __Chinese (Zh)→En__ (prepared corpora is be provided):

1. Recurrent neural network based encoder-decoder without attention
2. Recurrent neural network based encoder-decoder with attention
2. Replace the recurrent encoder with either convolutional or self-attention based encoder.
4. [Optional] Build either or both fully self-attention translation system or/and multilingual translation system.

## Part 1: Data Upload & Preprocessing

In [4]:
# start of sentence
SOS_token = 1
# end of sentence
EOS_token = 3

## 2 = unk

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0:"<PAD>",1: "<SOS>", 2:"<UNK>", 3: "<EOS>"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [5]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    """About "NFC" and "NFD": 
    
    For each character, there are two normal forms: normal form C 
    and normal form D. Normal form D (NFD) is also known as canonical 
    decomposition, and translates each character into its decomposed form. 
    Normal form C (NFC) first applies a canonical decomposition, then composes 
    pre-combined characters again.
    
    About unicodedata.category: 
    
    Returns the general category assigned to the Unicode character 
    unichr as string."""
    
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Trim
def normalizeString(s):
    s = unicodeToAscii(s.strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [6]:
def readLangs(lang1, lang2, reverse=False,
             dataset="train"):
    
    """Takes as input;
    - lang1, lang2: either (vi, en) or (zh, en)
    - dataset: one of ("train","dev","test")"""
    print("Reading lines...")
    eos = [".","?","!","\n"]
    # Read the pretokenized lang1 file and split into lines
    lang1_lines = open("../data/tokens_and_preprocessing_em/pretokenized_data/iwslt-%s-%s-processed/%s.tok.%s" % (lang1, lang2, dataset, lang1), encoding="utf-8").\
        read().strip().split("\n")
    # Read the lang2 file and split into lines
    lang2_lines = open("../data/tokens_and_preprocessing_em/pretokenized_data/iwslt-%s-%s-processed/%s.tok.%s" % (lang1, lang2, dataset, lang2), encoding="utf-8").\
        read().strip().split("\n")
    
    # create sentence pairs (lists of length 2 that consist of string pairs)
    # e.g. ["And we &apos;re going to tell you some stories from the sea here in video .",
    #       "我们 将 用 一些 影片 来讲 讲述 一些 深海 海里 的 故事  "]
    # check if there are the same number of sentences in each set
    assert len(lang1_lines) == len(lang2_lines), "Two languages must have the same number of sentences. "+ str(len(lang1_lines)) + " sentences were passed for " + str(lang1) + "." + str(len(lang2_lines)) + " sentences were passed for " + str(lang2)+"."
    # normalize if not Chinese, Chinese normalization is already handeled
    if lang1 == "zh":
        lang1_lines = lang1_lines
    else:
        lang1_lines = [normalizeString(s) for s in lang1_lines]
    lang2_lines = [normalizeString(s) for s in lang2_lines]
    # construct pairs
    pair_ran = range(len(lang1_lines))
    pairs = [[lang1_lines[i]] + [lang2_lines[i]] for i in pair_ran]
    
#     # Split every line into pairs and normalize
#     pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [7]:
def prepareData(lang1, lang2, reverse=False, dataset="train"):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse, dataset=dataset)
    print("Read %s sentence pairs" % len(pairs))
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

# example
input_lang, output_lang, pairs = prepareData('vi', 'en', False, dataset="train")
print(random.choice(pairs))

Reading lines...
Read 133317 sentence pairs
Trimmed to 133317 sentence pairs
Counting words...
Counted words:
vi 16144
en 47568
['Tac pham nghe thuat tiep theo la Con chim moi .', 'The next work is Decoy .']


In [8]:
input_lang, output_lang, pairs = prepareData('zh', 'en', False, dataset="train")
print(random.choice(pairs))

Reading lines...
Read 213376 sentence pairs
Trimmed to 213376 sentence pairs
Counting words...
Counted words:
zh 88917
en 59329
['如果 比较 二者 的 特征 会 发现   享乐 视 时间   对象   地点 而 定  ', 'But if you look at the characteristics of those two pleasure is contingent upon time upon its object upon the place .']


### 1.1 Vietnamese to English

In [9]:
# # Please find the original tokenizing code provided by Elman Mansimov in the following link:
# # https://github.com/derincen/neural-machine-translation/tree/master/data/tokens_and_preprocessing_em/preprocess_translation

# def tokenize_vi(f_names, f_out_names):
#     for f_name, f_out_name in zip(f_names, f_out_names):
#         lines = open(f_name, 'r').readlines()
#         tok_lines = open(f_out_name, 'w')
#         for i, sentence in enumerate(lines):
#             if i > 0 and i % 1000 == 0:
#                 print (f_name.split('/')[-1], i, len(lines))
#             tok_lines.write(word_tokenize(sentence, format="text") + '\n')
#         tok_lines.close()

# def tokenize_en(f_names, f_out_names):
#     tokenizer = spacy.load('en_core_web_sm')

#     for f_name, f_out_name in zip(f_names, f_out_names):
#         lines = open(f_name, 'r').readlines()
#         tok_lines = open(f_out_name, 'w')
#         for i, sentence in enumerate(lines):
#             if i > 0 and i % 1000 == 0:
#                 print (f_name.split('/')[-1], i, len(lines))
#             # replaced tokenizer(sentence) with str(tokenizer(sentence)) to avoid 
#             # type error while joining
#             tok_lines.write(' '.join(str(tokenizer(sentence))) + '\n')
#         tok_lines.close()


# if __name__ == "__main__":
#     root = '../data/tokens_and_preprocessing_em/pretokenized_data/iwslt-vi-en-processed/'
#     tokenize_vi([os.path.join(root, 'train.vi'), os.path.join(root, 'dev.vi'), 
#                  os.path.join(root, 'test.vi')],\
#                [os.path.join(root, 'train.tok.vi'), os.path.join(root, 'dev.tok.vi'), 
#                 os.path.join(root, 'test.tok.vi')])

#     tokenize_en([os.path.join(root, 'train.en'), os.path.join(root, 'dev.en'), 
#                  os.path.join(root, 'test.en')],\
#                 [os.path.join(root, 'train.tok.en'), os.path.join(root, 'dev.tok.en'), 
#                  os.path.join(root, 'test.tok.en')])


In [10]:
# Format: languagepair_language_dataset
# Train 
vien_vi_train, vien_en_train, vi_en_train_pairs = prepareData('vi', 'en', False, dataset="train")
# Dev 
vien_vi_dev, vien_en_dev, vi_en_dev_pairs = prepareData('vi', 'en', False, dataset="dev")
# Test
vien_vi_test, vien_en_test, vi_en_test_pairs = prepareData('vi', 'en', False, dataset="test")

Reading lines...
Read 133317 sentence pairs
Trimmed to 133317 sentence pairs
Counting words...
Counted words:
vi 16144
en 47568
Reading lines...
Read 1268 sentence pairs
Trimmed to 1268 sentence pairs
Counting words...
Counted words:
vi 1370
en 3816
Reading lines...
Read 1553 sentence pairs
Trimmed to 1553 sentence pairs
Counting words...
Counted words:
vi 1325
en 3619


### 1.2 Chinese to English

In [11]:
# # Please find the original tokenizing code provided by Elman Mansimov in the following link:
# # https://github.com/derincen/neural-machine-translation/tree/master/data/tokens_and_preprocessing_em/preprocess_translation

# def tokenize_zh(f_names, f_out_names):
#     for f_name, f_out_name in zip(f_names, f_out_names):
#         lines = open(f_name, 'r').readlines()
#         tok_lines = open(f_out_name, 'w')
#         for i, sentence in enumerate(lines):
#             if i > 0 and i % 1000 == 0:
#                 print (f_name.split('/')[-1], i, len(lines))
#             tok_lines.write(' '.join(jieba.cut(sentence, cut_all=True)))
#         tok_lines.close()

# def tokenize_en(f_names, f_out_names):
#     tokenizer = spacy.load('en_core_web_sm')

#     for f_name, f_out_name in zip(f_names, f_out_names):
#         lines = open(f_name, 'r').readlines()
#         tok_lines = open(f_out_name, 'w')
#         for i, sentence in enumerate(lines):
#             if i > 0 and i % 1000 == 0:
#                 print (f_name.split('/')[-1], i, len(lines))
#             # replaced tokenizer(sentence) with str(tokenizer(sentence)) to avoid 
#             # type error while joining
#             tok_lines.write(' '.join(str(tokenizer(sentence))) + '\n')
#         tok_lines.close()

# if __name__ == "__main__":
#     root = '../data/tokens_and_preprocessing_em/pretokenized_data/iwslt-zh-en-processed/'
#     tokenize_zh([os.path.join(root, 'dev.zh'), os.path.join(root, 'test.zh'), os.path.join(root, 'train.zh')],\
#                 [os.path.join(root, 'dev.tok.zh'), os.path.join(root, 'test.tok.zh'), os.path.join(root, 'train.tok.zh')])

# #     tokenize_en([os.path.join(root, 'dev.en'), os.path.join(root, 'test.en'), os.path.join(root, 'train.en')],\
# #                [os.path.join(root, 'dev.tok.en'), os.path.join(root, 'test.tok.en'), os.path.join(root, 'train.tok.en')])


In [12]:
# Format: languagepair_language_dataset
# Train 
zhen_zh_train, zhen_en_train, zh_en_train_pairs = prepareData('zh', 'en', False, dataset="train")
# Dev 
zhen_zh_dev, zhen_en_dev, zh_en_dev_pairs = prepareData('zh', 'en', False, dataset="dev")
# Test
zhen_zh_test, zhen_en_test, zh_en_test_pairs = prepareData('zh', 'en', False, dataset="test")

Reading lines...
Read 213376 sentence pairs
Trimmed to 213376 sentence pairs
Counting words...
Counted words:
zh 88917
en 59329
Reading lines...
Read 1261 sentence pairs
Trimmed to 1261 sentence pairs
Counting words...
Counted words:
zh 6132
en 3916
Reading lines...
Read 1397 sentence pairs
Trimmed to 1397 sentence pairs
Counting words...
Counted words:
zh 5214
en 3423


In [13]:
zh_en_train_pairs[3]

['我们 将 用 一些 影片 来讲 讲述 一些 深海 海里 的 故事  ',
 'And we apos re going to tell you some stories from the sea here in video .']

### 1.3: Check Source & Target Vocabs

Since the source and target languages can have very different table lookup layers, it's good practice to have separate vocabularies for each. Thus, we build vocabularies for each language that we will be using. 

In the first class (Lang) of this section, we have already defined vocabularies for all languages. So, there is no need to redefine another function. We chech each vocabulary below.

#### Chinese Vocabulary

In [14]:
print ("The number of words in Chinese training corpus is " + str(zhen_zh_train.n_words))

The number of words in Chinese training corpus is 88917


In [15]:
zhen_zh_train.word2index["格"]

10479

In [16]:
zhen_zh_train.index2word[10479]

'格'

#### Vietnamese Vocabulary

In [17]:
print ("The number of words in Vietnamese training corpus is " + str(vien_vi_train.n_words))

The number of words in Vietnamese training corpus is 16144


In [18]:
vien_vi_train.word2index["Hamburger"]

6752

In [19]:
vien_vi_train.index2word[6752]

'Hamburger'

#### English Vocabulary for Zh-En

In [20]:
print ("The number of words in English training corpus for Zh-En is " + str(zhen_en_train.n_words))

The number of words in English training corpus for Zh-En is 59329


In [21]:
zhen_en_train.word2index["translate"]

1451

In [22]:
zhen_en_train.index2word[1451]

'translate'

#### English Vocabulary for Vi-En

In [23]:
print ("The number of words in English training corpus for Vi-En is " + str(vien_en_train.n_words))

The number of words in English training corpus for Vi-En is 47568


In [24]:
vien_en_train.word2index["machine"]

847

In [25]:
vien_en_train.index2word[847]

'machine'

### 1.4 Prepare Dataloaders

In [26]:
PAD_IDX = 0
SOS_IDX = 1
UNK_IDX = 2
EOS_IDX = 3
# convert token to id in the dataset
def token2index_dataset(paired_tokens, 
                        lang1_token2id_vocab,
                        lang2_token2id_vocab):
    """Takes as input:
    - paired_tokens: a list of sentence pairs that consist of source & target lang sentences.
    - lang1_token2id_vocab: token2index vocabulary for the first language. 
                            Get by method Lang_dataset.word2index
    - lang2_token2id_vocab: token2index vocabulary for the second language. 
                            Get by method Lang_dataset.word2index
                            
    Returns:
    - indices_data_lang_1, indices_data_lang2: A list of lists where each sub-list holds corresponding indices for each
                                               token in the sentence."""
    indices_data_lang_1, indices_data_lang_2 = [], []
    vocabs = [lang1_token2id_vocab, lang2_token2id_vocab]
    
    # lang1
    for t in range(len(paired_tokens)):
        index_list = [SOS_IDX] + [vocabs[0][token] if token in vocabs[0]\
                                    else UNK_IDX for token in paired_tokens[t][0]] + [EOS_IDX]
        indices_data_lang_1.append(index_list)
    # lang2
    for t in range(len(paired_tokens)):
        index_list = [SOS_IDX] + [vocabs[1][token] if token in vocabs[1] \
                                    else UNK_IDX for token in paired_tokens[t][1]] + [EOS_IDX]
        indices_data_lang_2.append(index_list)
        
    return indices_data_lang_1, indices_data_lang_2

# train indices
zhen_zh_train_indices, zhen_en_train_indices = token2index_dataset(zh_en_train_pairs,
                                                                   zhen_zh_train.word2index,
                                                                   zhen_en_train.word2index)

vien_vi_train_indices, vien_en_train_indices = token2index_dataset(vi_en_train_pairs,
                                                                   vien_vi_train.word2index,
                                                                   vien_en_train.word2index)

# dev indices
zhen_zh_dev_indices, zhen_en_dev_indices = token2index_dataset(zh_en_dev_pairs,
                                                               zhen_zh_dev.word2index,
                                                               zhen_en_dev.word2index)

vien_vi_dev_indices, vien_en_dev_indices = token2index_dataset(vi_en_dev_pairs,
                                                               vien_vi_dev.word2index,
                                                               vien_en_dev.word2index)

# test indices
zhen_zh_test_indices, zhen_en_test_indices = token2index_dataset(zh_en_test_pairs,
                                                                 zhen_zh_test.word2index,
                                                                 zhen_en_test.word2index)

vien_vi_test_indices, vien_en_test_indices = token2index_dataset(vi_en_test_pairs,
                                                                 vien_vi_test.word2index,
                                                                 vien_en_test.word2index)

In [27]:
EOS_IDX

3

In [28]:
# check length
# train
print ("Chinese training set length = "+str(len(zhen_zh_train_indices)))
print ("Chinese-English (En) training set length = "+str(len(zhen_en_train_indices)))
print ("\nVietnamese training set length = "+str(len(vien_vi_train_indices)))
print ("Vietnamese-English (En) training set length = "+str(len(vien_en_train_indices)))
# dev
print ("\nChinese dev set length = "+str(len(zhen_zh_dev_indices)))
print ("Chinese-English (En) dev set length = "+str(len(zhen_en_dev_indices)))
print ("\nVietnamese dev set length = "+str(len(vien_vi_dev_indices)))
print ("Vietnamese-English (En) dev set length = "+str(len(vien_en_dev_indices)))
# test
print ("\nChinese test set length = "+str(len(zhen_zh_test_indices)))
print ("Chinese-English (En) test set length = "+str(len(zhen_en_test_indices)))
print ("\nVietnamese test set length = "+str(len(vien_vi_test_indices)))
print ("Vietnamese-English (En) test set length = "+str(len(vien_en_test_indices)))

Chinese training set length = 213376
Chinese-English (En) training set length = 213376

Vietnamese training set length = 133317
Vietnamese-English (En) training set length = 133317

Chinese dev set length = 1261
Chinese-English (En) dev set length = 1261

Vietnamese dev set length = 1268
Vietnamese-English (En) dev set length = 1268

Chinese test set length = 1397
Chinese-English (En) test set length = 1397

Vietnamese test set length = 1553
Vietnamese-English (En) test set length = 1553


#### Dataloader

In [29]:
UNK_IDX

2

In [30]:
from torch.utils.data import Dataset

In [31]:
len(zhen_zh_train.word2index)

88915

In [32]:
## TODO 

MAX_SENTENCE_LENGTH = 300
BATCH_SIZE = 32

# zhen token2index vocabs
zhen_zh_train_token2id = zhen_zh_train.word2index
zhen_en_train_token2id = zhen_en_train.word2index

# vien token2index vocabs
vien_vi_train_token2id = vien_vi_train.word2index
vien_en_train_token2id = vien_en_train.word2index

class TranslationDataset():
    """
    Class that represents a train/dev/test dataset that's readable for PyTorch
    Note that this class inherits torch.utils.data.Dataset
    """

    def __init__(self, 
                 data_source, # training indices data of the source language
                 data_target, # training indices data of the target language
                 token2id_source=None, # token2id dict of the source language
                 token2id_target=None  # token2id dict of the target language
                ):
        """
        @param data_list: list of character
        @param target_list: list of targets

        """
        self.source_sentences, self.target_sentences =  data_source, data_target
        
        self.token2id_source = token2id_source
        self.token2id_target = token2id_target

    def __len__(self):
        return len(self.source_sentences)

    def __getitem__(self, batch_index):

#         source_word_idx, target_word_idx = [], []
        source_mask, target_mask = [], []
        
        for index in self.source_sentences[batch_index][:MAX_SENTENCE_LENGTH]:
            if index != UNK_IDX:
                source_mask.append(0)
            else:
                source_mask.append(1)
                
        for index in self.target_sentences[batch_index][:MAX_SENTENCE_LENGTH]:
            if index != UNK_IDX:
                target_mask.append(0)
            else:
                target_mask.append(1)
        
        source_indices = [SOS_token] + self.source_sentences[batch_index][:MAX_SENTENCE_LENGTH]
        target_indices = [SOS_token] + self.target_sentences[batch_index][:MAX_SENTENCE_LENGTH]
        
        source_list = [source_indices, source_mask, len(source_indices)]
        target_list = [target_indices, target_mask, len(target_indices)]
        
        return source_list + target_list

    
def translation_collate(batch, max_sentence_length):
    """
    Customized function for DataLoader that dynamically pads the batch so that all
    data have the same length
    """
    source_data, target_data = [], []
    source_mask, target_mask = [], []
    source_lengths, target_lengths = [], []

    for datum in batch:
        source_lengths.append(datum[2])
        target_lengths.append(datum[5])
        
        # PAD
        source_data_padded = np.pad(np.array(datum[0]), pad_width=((0, MAX_SENTENCE_LENGTH-datum[2])),
                                mode="constant", constant_values=0)
        source_data.append(source_data_padded)
        
        source_mask_padded = np.pad(np.array(datum[1]), pad_width=((0, MAX_SENTENCE_LENGTH-datum[2])),
                                mode="constant", constant_values=0)
        source_mask.append(source_mask_padded)
        
        target_data_padded = np.pad(np.array(datum[3]), pad_width=((0, MAX_SENTENCE_LENGTH-datum[5])),
                                mode="constant", constant_values=0)
        target_data.append(target_data_padded)
        
        target_mask_padded = np.pad(np.array(datum[4]), pad_width=((0, MAX_SENTENCE_LENGTH-datum[5])),
                               mode="constant", constant_values=0)
        target_mask.append(target_mask_padded)
        
    ind_dec_order = np.argsort(source_lengths)[::-1]
    source_data = np.array(source_data)[ind_dec_order]
    target_data = np.array(target_data)[ind_dec_order]
    source_mask = np.array(source_mask)[ind_dec_order].reshape(len(batch), -1, 1)
    target_mask = np.array(target_mask)[ind_dec_order].reshape(len(batch), -1, 1)
    source_lengths = np.array(source_lengths)[ind_dec_order]
    target_lengths = np.array(target_lengths)[ind_dec_order]
    
    source_list = [torch.from_numpy(source_data), 
               torch.from_numpy(source_mask).float(), source_lengths]
    target_list = [torch.from_numpy(target_data), 
               torch.from_numpy(target_mask).float(), target_lengths]
        
    return source_list + target_list


zhen_train_dataset = TranslationDataset(zhen_zh_train_indices,
                                       zhen_en_train_indices,
                                       token2id_source=zhen_zh_train_token2id,
                                       token2id_target=zhen_en_train_token2id)

zhen_train_loader = torch.utils.data.DataLoader(dataset=zhen_train_dataset,
                               batch_size=BATCH_SIZE,
                               collate_fn=lambda x, max_sentence_length=MAX_SENTENCE_LENGTH: translation_collate(x, MAX_SENTENCE_LENGTH),
                               shuffle=False)

zhen_dev_dataset = TranslationDataset(zhen_zh_dev_indices,
                                       zhen_en_dev_indices,
                                       token2id_source=zhen_zh_train_token2id,
                                       token2id_target=zhen_en_train_token2id)

zhen_dev_loader = torch.utils.data.DataLoader(dataset=zhen_dev_dataset,
                             batch_size=BATCH_SIZE,
                             collate_fn=lambda x, max_sentence_length=MAX_SENTENCE_LENGTH: translation_collate(x, MAX_SENTENCE_LENGTH),
                             shuffle=False)

vien_train_dataset = TranslationDataset(vien_vi_train_indices,
                                       vien_en_train_indices,
                                       token2id_source=vien_vi_train_token2id,
                                       token2id_target=vien_en_train_token2id)

vien_train_loader = torch.utils.data.DataLoader(dataset=vien_train_dataset,
                             batch_size=BATCH_SIZE,
                             collate_fn=lambda x, max_sentence_length=MAX_SENTENCE_LENGTH: translation_collate(x, MAX_SENTENCE_LENGTH),
                             shuffle=False)

vien_dev_dataset = TranslationDataset(vien_vi_dev_indices,
                                       vien_en_dev_indices,
                                       token2id_source=vien_vi_train_token2id,
                                       token2id_target=vien_en_train_token2id)

vien_dev_loader = torch.utils.data.DataLoader(dataset=vien_dev_dataset,
                             batch_size=BATCH_SIZE,
                             collate_fn=lambda x, max_sentence_length=MAX_SENTENCE_LENGTH: translation_collate(x, MAX_SENTENCE_LENGTH),
                             shuffle=False)




In [33]:
# [*vien_dev_loader][0]

## Part 2: Evaluation Metric

We use BLEU as the evaluation metric. Specifically, we focus on the corpus-level BLEU function. 

The code for BLEU is taken from https://github.com/mjpost/sacreBLEU/blob/master/sacrebleu.py#L1022-L1080

In [34]:
! pip3 install sacrebleu

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [35]:
import sacrebleu

ModuleNotFoundError: No module named 'sacrebleu'

## Part 3: Beam Search Algorithm

In this section, we implement the Beam Search algorithm in Pytorch.

In [ ]:
# initialize k-many score lists
# start only with the whole x
# initialize k-many prev y's lists
# choose top-k for y1 from the whole vocab
# choose top-k for the second time step by expanding the first time step
# compute scores by adding log probabilities

In [ ]:
# beam_size_k = 10

# class BeamSearch:
    
#     """RECURSE"""
    
#     def __init__(self,
#                  beam_size=beam_size_k, ## insert num 
#                  softmax_out
#                 ):
#         """
#         Class that holds beam information, and search & score functions
#         - beam_size = beam size
#         - softmax_out = the softmax over the vocabulary at time step t, as computed by the RNN decoder,
#                         given the source sequence X and the previously decoded y_<t tokens.
#         """
        
#         self.beam_size = beam_size
#         self.softmax_out = softmax_out
        
#         # initialize paths
#         self.paths = np.empty((self.beam_size))
        
#         # initialize the dictionary that will hold the path scores 
#         # and update the scores at each time step
#         self.path_score_dict = {}
#         # we will later use each i < k as a key and populate this
#         # dict with scores

        
#     def search():
        
        
        
    
#     def score(prev_ys = None):
#         """- prev_ys = previously decoded tokens (previously generated target language tokens)
#         """
        
    

## Part 4: Model

1. Recurrent neural network based encoder-decoder without attention
2. Recurrent neural network based encoder-decoder with attention
2. Replace the recurrent encoder with either convolutional or self-attention based encoder.

#### Loss Function & Evaluation

In [36]:
# reconstruction loss = binary cross entropy between two (vocab_size x 1) vectors
# used during training, since we can compare the real Y and and the generated Y
# still at each time step of the decoder, we compare up to and including
# the real t-th token and the generated t-th, then optimize

def loss_function(y_hat, y):
    
    """Takes as input;
    - y: correct "log-softmax"(binary vector) that represents the correct t-th token in the target sentence,
                 (vocab_size x 1) vector
    - y_hat: predicted LogSoftmax for the predicted t-th token in the target sentence.
             (vocab_size x 1) vector
    Returns;
    - NLL Loss in training time"""
#     y_hat = torch.log(y_hat) # log softmax
    loss = nn.functional.binary_cross_entropy(y_hat,y)
    
    return loss
    

# generation/inference time - validation loss = BLEU

def compute_BLEU(corpus_hat,corpus):
    ## TODO
    return None


#### Beam Search

In [37]:
# MAX_PATH_LENGTH = 400 # make changeable later !!!

# class TargetOut:
#     def __init__(self,
#                  beam_size=5,
#                  source_sentence_length=400,
#                  time_step=0):
#         """
#         - beam: the tensor that will be populated with beam_size-many paths in each timestep
#         - beam_size: the width of the beam, top k tokens to include in the beam search,
#         """
        
#         # initialized again for each timestep
#         self.beam = torch.empty(beam_size)
#         self.beam_size = beam_size
#         self.beam_seq = beam_seq
#         self.time_step = time_step
        
#         self.max_target_length = source_sentence_length*(1.5)
#         # path is kept by hold_path
#         self.path = torch.empty(beam_size, max_target_length)
    
#     def _add_and_score_paths(self, 
#              top_k_tokens):
        
#         """top_k_tokens: torch.FloatTensor of indices according to logSoftmax 
#         (not embeddings - embedding matrix indices or vocab indices)"""
        
#         time_step = self.time_step
#         self.path[:,time_step] = top_k_tokens
        
#         return self
            
#     def _score_paths(self,gru_out):
        
#         """For each path, computes log(P(Y_i|Y_i-1,..,Y_i-n,X)) + log(P(Y_i-1|Y_i-2,..,Y_i-n,X)) + ...
#         -gru_out is a softmax over the vocabulary for each timestep, so 
#         we need to take its log to obtain the scores"""
#         if self.time_step = 0:
            
        
    
#     def _hold_path_score(self):
        
        





### 2.1: RNN-based Encoder-Decoder without Attention

In [38]:
batch_size = 32

In [39]:
torch.cat([torch.FloatTensor([2]).view(-1,1),torch.FloatTensor([3]).view(-1,1)],dim=1)

tensor([[2., 3.]])

In [40]:
nn.ReLU([])

ReLU()

In [105]:
# from https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

class RNNencoder(nn.Module):
    def __init__(self,
                 vocab_size=len(zhen_zh_train_token2id), # for chinese
                 embedding_size=300,
                 percent_dropout=0.3, 
                 hidden_size=256,
                 num_gru_layers=2):
        
        super(RNNencoder, self).__init__()
        
        self.vocab_size = vocab_size
        self.embed_size = embedding_size
        self.dropout = percent_dropout
        self.embed_source = nn.Embedding(self.vocab_size,
                                         self.embed_size,
                                         padding_idx=0
                                        )

        self.hidden_size = hidden_size
        self.num_layers = num_gru_layers
        
        self.GRU = nn.GRU(self.embed_size, 
                          self.hidden_size, 
                          self.num_layers, 
                          batch_first=True, bidirectional=False)
        
        self.drop_out_function = nn.Dropout(self.dropout)
        
    def init_hidden(self, batch_size):
        hidden_ = torch.zeros(self.num_layers, ## 2 for bidirectional
                             batch_size, self.hidden_size).to(device)
        return hidden_

    def forward(self, source_sentence, source_mask, source_lengths):
        """Returns source lengths to feed into the decoder, since we do not want
        the translation length to be above/below a certain treshold*source sentence length."""
        
        sort_original_source = sorted(range(len(source_lengths)), 
                             key=lambda sentence: -source_lengths[sentence])
        unsort_to_original_source = sorted(range(len(source_lengths)), 
                             key=lambda sentence: sort_original_source[sentence])
        
        source_sentence = source_sentence[sort_original_source]
        _source_mask = source_mask[sort_original_source]
        source_lengths = source_lengths[sort_original_source]
        batch_size, seq_len_source = source_sentence.size()
        # init hidden
        self.hidden_source = self.init_hidden(batch_size)
        embeds_source = self.embed_source(source_sentence)
        embeds_source = source_mask*embeds_source + (1-_source_mask)*embeds_source.clone().detach()
        
        embeds_source = torch.nn.utils.rnn.pack_padded_sequence(embeds_source, 
                                                                source_lengths, 
                                                                batch_first=True)
        
        gru_out_source, self.hidden_source = self.GRU(embeds_source, self.hidden_source)
        
        # ref: pytorch documentation
        # hidden source : h_n of shape 
        # (num_layers * num_directions, batch_size, hidden_size)
#         print ("hidden source size = "+str(self.hidden_source.size()))
        
        # ref: pytorch documentation
        # Like output, the layers can be separated using 
        # h_n.view(num_layers, num_directions, batch_size, hidden_size)
        if self.GRU.bidirectional:
            self.num_directions = 2
        else:
            self.num_directions = 1
            
        hidden_source = self.hidden_source.view(self.num_layers, self.num_directions, 
                                                batch_size, self.hidden_size)
        
        # the following should print (2, 1, 32, 256) for this config
#         print ("hidden source size after view = "+str(hidden_source.size()))
        
        # get the mean along 0th axis (over layers)
        hidden_source = torch.mean(hidden_source, dim=0) ## mean instead of sum for source representation as suggested in the class
        # the following should print (1, 32, 256)
#         print ("hidden source size after mean = "+str(hidden_source.size()))
        
        if self.GRU.bidirectional:
            hidden_source = torch.cat([hidden_source[:,i,:] for i in range(self.num_directions)], dim=1)
        else:
            hidden_source = hidden_source
            
        # view before unsort
        hidden_source = hidden_source.view(batch_size, self.hidden_size)
        
        # the following should print (32, 256)
#         print("hidden source size before unsort = "+str(hidden_source.size()))
        hidden_source = hidden_source[unsort_to_original_source] ## back to original indices
#         print ("ENCODER hidden source size after unsort = "+str(hidden_source.size()))
        
        gru_out_source, _ = torch.nn.utils.rnn.pad_packed_sequence(gru_out_source,
                                                                  batch_first=True)
        
        gru_out_source = gru_out_source.view(batch_size, -1, 2, self.hidden_size)
        gru_out_source = torch.mean(gru_out_source, dim=1) ## mean instead of sum for source representation as suggested in the class
        gru_out_source = torch.cat([gru_out_source[:,i,:] for i in range(2)], dim=1)
        gru_out_source = gru_out_source[unsort_to_original_source] ## back to original indices
        
        source_lengths = source_lengths[unsort_to_original_source]

        return hidden_source, source_lengths
    

In [160]:
# from https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

class RNNdecoder(nn.Module):
    def __init__(self,
                 vocab_size=len(zhen_en_train_token2id), # for chinese-english's english
                 embedding_size=300,
                 percent_dropout=0.3, 
                 hidden_size=256,
                 num_gru_layers=1,
                 max_sentence_len=300):
        
        super(RNNdecoder, self).__init__()
        self.vocab_size = vocab_size
        self.embed_size = embedding_size
        self.dropout = percent_dropout
        self.max_sentence_len = max_sentence_len

        self.hidden_size = hidden_size
        self.num_layers = num_gru_layers
        
        self.GRU = nn.GRU(self.embed_size, 
                          self.hidden_size, 
                          self.num_layers, 
                          batch_first=True, bidirectional=False)
        
        self.ReLU = nn.ReLU
        
        self.drop_out_function = nn.Dropout(self.dropout)
        
        self.embed_target = nn.Embedding(self.vocab_size,
                                         self.embed_size, padding_idx=0)
        
        self.sigmoid = nn.Sigmoid()
        
        # *2 because we are concating hidden with embedding plus context
        self.linear_layer = nn.Linear(self.hidden_size*2, self.vocab_size)
        self.log_softmax = nn.LogSoftmax(dim=0)
        self.softmax = nn.Softmax(dim=0)
        
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, 
                             batch_size, self.hidden_size).to(device)
        
        return hidden

    def forward(self,
                hidden_source, ## context 
                previously_decoded_upto_t, # input
                hidden, # initial hidden (will pass hidden source - same as context)
                target_lengths,
                target_mask,
                time_step):
        
        # input = previously decoded y tokens = <SOS> token for first token
        self.input = previously_decoded_upto_t

        sort_original_target = sorted(range(len(target_lengths)), 
                             key=lambda sentence: -target_lengths[sentence])
        unsort_to_original_target = sorted(range(len(target_lengths)), 
                             key=lambda sentence: sort_original_target[sentence])

        self.input = self.input[sort_original_target]
        _target_mask = target_mask[sort_original_target]
        target_lengths = target_lengths[sort_original_target]
        batch_size, seq_len_target = self.input.size()
        
        if self.GRU.bidirectional:
            self.num_directions = 2
        else:
            self.num_directions = 1
        
        # context = hidden_source
        self.context = hidden_source.view(batch_size, 
                                          self.num_directions,
                                          self.hidden_size)
            
        # hidden => initial hidden will be the same as the context
        # vector, which is the hidden_source tensor
        self.hidden = hidden.view(self.num_layers*self.num_directions,
                                  batch_size, self.hidden_size)
        
        # embed target
        embeds_target = self.drop_out_function(self.embed_target(self.input.long()))

        embeds_target = target_mask*embeds_target + (1-_target_mask)*embeds_target.clone().detach()
#         print ("------------------------")
#         print ("embedding size before concat = "+str(embeds_target.size()))
        
        self.context = self.context.repeat(1,self.embed_size,1)
#         print ("context vector size before concat = "+str(self.context.size()))
#         print ("input size before concat = "+str(self.input.size()))
        concat_embed = torch.cat([embeds_target, self.context], dim=2)
#         print ("concat embed size = "+str(concat_embed.size()))
        
        embeds_target = torch.nn.utils.rnn.pack_padded_sequence(embeds_target,
                                                                target_lengths,
                                                                batch_first=True)
        
        
        gru_out_target, self.hidden = self.GRU(embeds_target, self.hidden)
        
        # ref: pytorch documentation
        # hidden source : h_n of shape 
        # (num_layers * num_directions, batch_size, hidden_size)
        # the following should print (1, 32, 256) for this config
#         print ("hidden size after GRU = "+str(self.hidden.size()))
        
        # undo packing
        gru_out_target, _ = torch.nn.utils.rnn.pad_packed_sequence(gru_out_target,
                                                                   batch_first=True)
        
#         print ("gru out size after GRU = "+str(gru_out_target.size()))
        
        hidden = self.hidden.view(self.num_layers, self.num_directions,
                                  batch_size, self.hidden_size)
        
        gru_out_target = gru_out_target.contiguous().view(self.max_sentence_len, 
                                             batch_size, 
                                             self.num_directions, 
                                             self.hidden_size)
        
        # the following should print (1, 1, 32, 256) for this config
#         print ("hidden size after view = "+str(hidden.size()))
        # the following should print (300, 32, 1, 256)
#         print ("gru out target size after view = "+str(gru_out_target.size()))
        
        hidden = torch.sum(hidden, dim=0) # we don't divide here, just sum
        # sum along sequence
        gru_out_target = torch.sum(gru_out_target, dim=0) # we don't divide here, just sum
        
        # the following should print (32, 1, 256)
#         print ("gru out target size after sum = "+ str(gru_out_target.size()))
        # the following should print (1, 32, 256)
#         print ("hidden size after sum = "+str(hidden.size()))
        
        if self.GRU.bidirectional:
            hidden = torch.cat([hidden[:,i,:] for i in range(self.num_directions)], 
                               dim=0)
            gru_out_target = torch.cat([gru_out_target[:,i,:] for i in range(self.num_directions)], 
                                       dim=1)
        else:
            hidden = hidden.view(batch_size, 
                                 self.num_directions, self.hidden_size)
            gru_out_target = gru_out_target.view(batch_size,
                                                 self.num_directions, self.hidden_size)
        
        # the following two should both print (32, 256)
#         print ("hidden size before unsort = "+str(hidden.size()))
#         print ("gru out target size before unsort = "+str(gru_out_target.size()))
        
        hidden = hidden[unsort_to_original_target] ## back to original indices
        gru_out_target = gru_out_target[unsort_to_original_target] ## back to original indices

        # the following two should both print (32, 256)
#         print ("hidden size after unsort = "+str(hidden.size()))
#         print ("gru out target size after unsort = "+str(gru_out_target.size()))

        gru_out_target = self.sigmoid(gru_out_target)
        # concating embedding + context = gru_out_target with hidden
        out = torch.cat([gru_out_target,hidden], dim=2)
        
#         print ("out size after concat = "+str(out.size()))
        
        out = self.linear_layer(out)
        
#         print ("out size after linear layer = "+str(out.size()))
        
        # softmax over vocabulary
        pred = self.log_softmax(out)

        return pred, hidden

In [161]:
def convert_to_softmax(tensor_of_indices,
                      vocab_size = len(zhen_en_train_token2id)):
    """
    - tensor_of_indices: tensor of size (batch_size)
    
    Returns: tensor of size (batch_size x vocab size) that
             represents the softmax over vocabulary for the true tokens
             at each timestep for each sentence in the batch."""
    
#     inp = torch.LongTensor(1, 32) % 100
    index_tensor_ = torch.unsqueeze(tensor_of_indices, 2)

    one_hot = torch.FloatTensor(1, 32, vocab_size).zero_()
    one_hot.scatter_(2, index_tensor_, 1)
    
    return one_hot

In [162]:
class Translate(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, source_sentence, target_sentence, 
                source_mask, target_mask, source_lengths,
                target_lengths):

        # to hold previously decoded ys
        y_outputs = torch.zeros(batch_size, 
                                target_sentence.size(1), 
                                len(zhen_en_train_token2id)).to(device)
        
        print ("y_outputs size = "+str(y_outputs.size()))
#         print (breaksss)
        #last hidden state of the encoder is the context
        context, source_lengths = self.encoder(source_sentence,
                                               source_mask,
                                               source_lengths)

        # context also used as the initial hidden state of the decoder
        decoder_hidden = context

        # # decoder should start with SOS tokens 
        # ref: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
        input = target_sentence[:,0].view(-1,1)
#         print ("input = "+str(input))
#         print ("input size = "+str(input.size()))
        
        for t in range(1, target_sentence.size(1)):
            
            decoder_out, decoder_hidden = self.decoder(context, # gru_out_source - instead of encoded_source[0]
                                                 input, # instead of target sentence up to t 
                                                 decoder_hidden,
                                                 target_lengths,  # target lengths
                                                 target_mask,
                                                 t)
            
#             print ("decoder out size = "+str(decoder_out.size()))
            for s in range(batch_size):
                y_outputs[s,t-1] = decoder_out[s,0]
#             print ("y_outputs size = "+str(y_outputs.size()))
#             print ("decoder out = "+str(decoder_out))
#             print ("decoder out size = "+str(decoder_out.size()))
#             print ("decoder_out[s,0] = "+str(decoder_out[s,0]))

            token_out = torch.max(decoder_out.view(batch_size,self.decoder.vocab_size),1)[1]
#             print ("token out size = "+str(token_out.size()))
            print ("token out at time step t = "+str(token_out))
            input = token_out.view(-1,1)
            
        return y_outputs

In [163]:
# chinese -> english
enc = RNNencoder(vocab_size=len(zhen_zh_train_token2id), # for chinese
                 embedding_size=300,
                 percent_dropout=0.3, 
                 hidden_size=256,
                 num_gru_layers=2)

dec = RNNdecoder(vocab_size=len(zhen_en_train_token2id), # for chinese-english's english
                 embedding_size=300,
                 percent_dropout=0.3, 
                 hidden_size=256,
                 num_gru_layers=1)

model = Translate(enc, dec).to(device)

In [164]:
BATCH_SIZE = 32
def train(model, loader=zhen_train_loader,criterion=loss_function,
          optimizer=None, 
          epoch=None):
    
    model.train()
    
    epoch_loss = 0
    
    for batch_idx, (source_sentence, source_mask, source_lengths, 
                    target_sentence, target_mask, target_lengths)\
    in enumerate(loader):
        
        source_sentence, source_mask = source_sentence.to(device), source_mask.to(device),  
        target_sentence, target_mask = target_sentence.to(device), target_mask.to(device),
        
        optimizer.zero_grad()
        
        # output softmax as generated by decoder 
        output = model(source_sentence, target_sentence, 
                source_mask, target_mask, source_lengths,
                target_lengths)
        
        # real target token
        target = convert_to_softmax(target_sentence[:,t].view(1,target_sentence[:,t].size(0)))
        
        loss = criterion(out, target)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss/BATCH_SIZE

In [165]:

num_epochs = 3
lr = 1e-4
# batch_

loss_train = []

for epoch in range(num_epochs):
    print ("epoch = "+str(epoch))

    loss = train(model,
                       loader = zhen_train_loader,
                       optimizer = torch.optim.Adam(model.parameters(), 
                                                   lr=lr),
                      epoch = epoch)
    
    loss_train.append(loss)
    
    print (loss_train)

epoch = 0
y_outputs size = torch.Size([32, 300, 59327])
token out at time step t = tensor([14714, 10064,  2042,  4271, 56510, 49882,  6436, 23596,  2433, 40416,
        29259, 13277, 53454, 36690, 24355, 17092, 30626, 29536,  7056, 27282,
        54411, 57394,  1881, 21724, 42954, 28554,  2833, 21571, 48102, 12462,
        43285, 44228])
token out at time step t = tensor([54116, 47878, 31732, 33475, 43650, 52621, 49729, 31161, 20775, 14036,
        53681, 17673, 20463, 18203, 56931, 25397,  2067, 52235, 40142, 27697,
        14075, 24722, 41214, 14882, 23218, 33889, 48692, 34210, 18373, 46360,
        26753, 47316])
token out at time step t = tensor([  522, 51773, 38649, 45863,  7951, 17801, 31229, 50426, 16265, 43200,
        56245, 58044, 26280, 22160, 23734, 36323, 46856, 25713, 46829, 12879,
        37160, 55381, 55740, 13997, 25746, 11876,  8134, 26345, 58239, 47464,
          465, 15932])
token out at time step t = tensor([20249, 54678, 42686, 37024, 47907, 39851, 37466, 31970, 2

KeyboardInterrupt: 

In [ ]:
# BATCH_SIZE = 32
# def train(RNNenc=RNNencoder,
#           RNNdec=RNNdecoder,
#           generator=Linear_Layers, # Linear_Layers to generate next 1 token
#           loader=zhen_train_loader, # automate
#           optimizer=None, 
#           epoch=None):
    
#     RNNenc.train()
#     RNNdec.train()
#     generator.train()
#     total_loss = 0
    
#     # previously decoded upto t goes into decoder
    
#     for batch_idx, (source_sentence, source_mask, source_lengths, 
#                     target_sentence, target_mask, target_lengths)\
#     in enumerate(DataLoader):
            
#         source_sentence, source_mask = source_sentence.to(device), source_mask.to(device),  
#         target_sentence, target_mask = target_sentence.to(device), target_mask.to(device),
        
#         RNNenc.train()
#         RNNdec.train()
#         generator.train()
#         optimizer.zero_grad()
        
#         generated_target = torch.empty(0)
#         # decoder should start with SOS tokens 
#         # ref: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
#         prev_decoded_y_tokens = torch.zeros(BATCH_SIZE) + SOS_token
        
#         encoder_hidden, source_lengths = RNNenc(source_sentence,
#                                                         source_mask,
#                                                         source_lengths)
        
#         hidden_ = encoder_hidden
        
#         for t in range(target_sentence.size(1)):
            
#             print ("Time Step = "+str(t))
            
# #             target_sentence_upto_t = target_sentence[:,t].view(batch_size,t+1)
#             target_sentence_upto_t = target_sentence[:,:t]
#             print ("target_sentence_upto_t size = "+str(target_sentence_upto_t.size()))
            
#             decoder_out, decoder_hidden = RNNdec(encoder_hidden, # gru_out_source - instead of encoded_source[0]
#                                                  prev_decoded_y_tokens, # instead of target sentence up to t 
#                                                  hidden_,
#                                                  target_lengths,  # target lengths
#                                                  target_mask,
#                                                  t)

#             out = generator(decoder_out,t) # returns softmax, t for timestep
            
#             # vector embeddings
#             generated_target = torch.cat([generated_target, out], dim=1)
#             # tokens
#             print ("prev_decoded_y_tokens size = "+str(prev_decoded_y_tokens.size()))
#             print ("max out size = "+str(torch.max(out,1)[1].view(BATCH_SIZE,t+1).long().size()))
#             prev_decoded_y_tokens = torch.cat([prev_decoded_y_tokens.view(BATCH_SIZE,t+1).long(), torch.max(out,1)[1].view(BATCH_SIZE,t+1).long()], 
#                                               dim=1)
            
#             print ("prev_decoded_y_tokens: "+str(prev_decoded_y_tokens))
#             print ("prev_decoded_y_tokens size = "+str(prev_decoded_y_tokens.size()))
#             print ("target tokens to generate at time t (for each sentence in batch) : "+str(target_sentence[:,t]))
#             print ("model-generated tokens at time t : "+str(torch.max(out,1)[1]))
# #             print ("type torch max out = "+str(type(torch.max(out,1)[1])))
            
#             # only to calculate the loss at each time step t
#             target = convert_to_softmax(target_sentence[:,t].view(1,target_sentence[:,t].size(0)))
#             print ("target_size = "+str(target.size()))
            
#             loss = loss_function(out, target)
#             print ("Loss = "+str(loss))
# #             loss.cuda().backward()
#             loss.backward()
#             optimizer.step()

#             total_loss += loss.item() * len(source_sentence) / len(DataLoader.dataset)
        
# #         if (batch_idx+1) % (len(DataLoader.dataset)//(20*labels.shape[0])) == 0:
# #             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
# #                 epoch, (batch_idx+1) * labels.shape[0], len(DataLoader.dataset),
# #                 100. * (batch_idx+1) / len(DataLoader), loss.item()))

#     optimizer.zero_grad()
#     return total_loss


# # def test(RNN, 
# #          Linear_Classifier, 
# #          DataLoader, 
# #          criterion):

# #     RNN.eval()
# #     Linear_Classifier.eval()
    
# #     test_loss = 0
# #     label_list = []
# #     output_list = []
    
# #     with torch.no_grad():
# #         for batch_idx, (sentence1, s1_original, sentence1_lengths, 
# #                     sentence2, s2_original, sentence2_lengths, labels)\
# #                     in enumerate(DataLoader):

# #             sentence1, s1_original = sentence1.to(device), s1_original.to(device),  
# #             sentence2, s2_original = sentence2.to(device), s2_original.to(device),
# #             labels = labels.to(device)
            
# #             # Forward
# #             output_s1 = RNN(sentence1, 
# #                                   s1_original, 
# #                                   sentence1_lengths)
# #             # Reverse
# #             output_s2 = RNN(sentence2, 
# #                                   s2_original, 
# #                                   sentence2_lengths)
            
# #             out = Linear_Classifier(output_s1, output_s2)
        
# #             loss = criterion(out, labels)

# #             test_loss += loss.item()/len(DataLoader.dataset)

# #             output_list.append(out)
# #             label_list.append(labels)
            
# #             print ("outputs= "+str(torch.cat(output_list, dim=0)))
# #             print ("labels= "+str(torch.cat(label_list, dim=0)))
            
# #     return test_loss, torch.cat(output_list, dim=0), torch.cat(label_list, dim=0)

# # def accuracy(RNN, 
# #              Linear_Classifier, 
# #              DataLoader, 
# #              criterion):
    
# #     _, predicted, true_labels = test(RNN = RNN,
# #                               Linear_Classifier = Linear_Classifier,
# #                               DataLoader = DataLoader,
# #                               criterion = criterion)

# #     predicted = predicted.max(1)[1]
# #     return 100 * predicted.eq(true_labels.data.view_as(predicted)).float().mean().item()

In [ ]:
RNNencoder_model = RNNencoder() # encoder model
RNNdecoder_model = RNNdecoder() # decoder model and concatenation
generator_model = Linear_Layers() # logsoftmax over vocab
DataLoader = zhen_train_loader

num_epochs = 3
lr = 1e-4
# batch_

loss_train = []

for epoch in range(num_epochs):
    print ("epoch = "+str(epoch))

    loss = train(RNNencoder_model,
                 RNNdecoder_model,
                 generator_model,
                       loader = DataLoader,
                       optimizer = torch.optim.Adam(list(RNNencoder_model.parameters()) + \
                                                    list(RNNdecoder_model.parameters()) + \
                                                   list(generator_model.parameters()), 
                                                   lr=lr),
                      epoch = epoch)
    
    loss_train.append(loss)
    
    print (loss_train)

In [ ]:
torch.empty(0).unsqueeze(1) # only case where the unsquueze func cannot be executed 

In [ ]:
torch.randn(32,1).repeat(1,4)

In [ ]:
# weight tensor is empty - initialize weigths for rnn ? why doesn't this happen for the encoder ???

### 2.2 RNN-based Encoder-Decoder with Attention

In [55]:
# modified from https://github.com/yanwii/seq2seq/blob/master/seq2seq.py

###################
## ORIGINAL CODE ##
###################

# ENCODER

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1):
        super(EncoderRNN, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)

    def forward(self, word_inputs, hidden):
        seq_len = len(word_inputs)
        embedded = self.embedding(word_inputs).view(seq_len, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def init_hidden(self):
        hidden = Variable(torch.zeros(self.n_layers, 1, self.hidden_size))
        if USE_CUDA: hidden = hidden.cuda()
        return hidden
    
# ATTENTION
    
class Attn(nn.Module):
    def __init__(self, method, hidden_size, max_length):
        super(Attn, self).__init__()

        self.method = method
        self.hidden_size = hidden_size

        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)

        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.other = nn.Parameter(torch.FloatTensor(1, hidden_size))

    def forward(self, hidden, encoder_outputs):
        seq_len = len(encoder_outputs)

        attn_energies = Variable(torch.zeros(seq_len)) # B x 1 x S
        if USE_CUDA: attn_energies = attn_energies.cuda()

        for i in range(seq_len):
            attn_energies[i] = self.score(hidden, encoder_outputs[i])

        return F.softmax(attn_energies).unsqueeze(0).unsqueeze(0)

    def score(self, hidden, encoder_output):
        if self.method == 'dot':
            energy = torch.dot(hidden.view(-1), encoder_output.view(-1))
            return energy

        elif self.method == 'general':
            energy = self.attn(encoder_output)
            energy = torch.dot(hidden.view(-1), encoder_output.view(-1))
            return energy
        
# DECODER

class AttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, hidden_size, output_size, n_layers=1, dropout_p=0.1, max_length=10):
        super(AttnDecoderRNN, self).__init__()

        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size * 2, hidden_size, n_layers, dropout=dropout_p)
        self.out = nn.Linear(hidden_size * 2, output_size)

        if attn_model != 'none':
            self.attn = Attn(attn_model, hidden_size, self.max_length)

    def forward(self, word_input, last_context, last_hidden, encoder_outputs):

        word_embedded = self.embedding(word_input).view(1, 1, -1) # S=1 x B x N

        rnn_input = torch.cat((word_embedded, last_context.unsqueeze(0)), 2)
        rnn_output, hidden = self.gru(rnn_input, last_hidden)

        attn_weights = self.attn(rnn_output.squeeze(0), encoder_outputs)
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1)) # B x 1 x N

        rnn_output = rnn_output.squeeze(0) # S=1 x B x N -> B x N
        context = context.squeeze(1)       # B x S=1 x N -> B x N
        output = F.log_softmax(self.out(torch.cat((rnn_output, context), 1)))
        #output = self.out(torch.cat((rnn_output, context), 1))
        return output, context, hidden, attn_weights

In [ ]:
class seq2seq(nn.Module):
    def __init__(self):
        super(seq2seq, self).__init__()
        self.max_epoches = 100000
        self.batch_index = 0
        self.GO_token = 2
        self.EOS_token = 1
        self.input_size = 14
        self.output_size = 15
        self.hidden_size = 100
        self.max_length = 15
        self.show_epoch = 100
        self.use_cuda = USE_CUDA
        self.model_path = "./model/"
        self.n_layers = 1
        self.dropout_p = 0.05
        self.beam_search = True
        self.top_k = 5
        self.alpha = 0.5

        self.enc_vec = []
        self.dec_vec = []

        # 初始化encoder和decoder
        self.encoder = EncoderRNN(self.input_size, self.hidden_size, self.n_layers)
        self.decoder = AttnDecoderRNN('general', self.hidden_size, self.output_size, self.n_layers, self.dropout_p, self.max_length)

        if USE_CUDA:
            self.encoder = self.encoder.cuda()
            self.decoder = self.decoder.cuda()

        self.encoder_optimizer = optim.Adam(self.encoder.parameters())
        self.decoder_optimizer = optim.Adam(self.decoder.parameters())
        self.criterion = nn.NLLLoss()

    def loadData(self):
        with open("./data/enc.vec") as enc:
            line = enc.readline()
            while line:
                self.enc_vec.append(line.strip().split())
                line = enc.readline()

        with open("./data/dec.vec") as dec:
            line = dec.readline()
            while line:
                self.dec_vec.append(line.strip().split())
                line = dec.readline()

    def next(self, batch_size, eos_token=1, go_token=2, shuffle=False):
        inputs = []
        targets = []

        if shuffle:
            ind = random.choice(range(len(self.enc_vec)))
            enc = [self.enc_vec[ind]]
            dec = [self.dec_vec[ind]]
        else:
            if self.batch_index+batch_size >= len(self.enc_vec):
                enc = self.enc_vec[self.batch_index:]
                dec = self.dec_vec[self.batch_index:]
                self.batch_index = 0
            else:
                enc = self.enc_vec[self.batch_index:self.batch_index+batch_size]
                dec = self.dec_vec[self.batch_index:self.batch_index+batch_size]
                self.batch_index += batch_size
        for index in range(len(enc)):
            enc = enc[0][:self.max_length] if len(enc[0]) > self.max_length else enc[0]
            dec = dec[0][:self.max_length] if len(dec[0]) > self.max_length else dec[0]

            enc = [int(i) for i in enc]
            dec = [int(i) for i in dec]
            dec.append(eos_token)

            inputs.append(enc)
            targets.append(dec)

        inputs = Variable(torch.LongTensor(inputs)).transpose(1, 0).contiguous()
        targets = Variable(torch.LongTensor(targets)).transpose(1, 0).contiguous()
        if USE_CUDA:
            inputs = inputs.cuda()
            targets = targets.cuda()
        return inputs, targets

    def train(self):
        self.loadData()
        try:
            self.load_state_dict(torch.load(self.model_path+'params.pkl'))
        except Exception as e:
            print(e)
            print("No model!")
        loss_track = []

        for epoch in range(self.max_epoches):
            start = time.time()
            inputs, targets = self.next(1, shuffle=False)
            loss, logits = self.step(inputs, targets, self.max_length)
            loss_track.append(loss)
            _,v = torch.topk(logits, 1)
            pre = v.cpu().data.numpy().T.tolist()[0][0]
            tar = targets.cpu().data.numpy().T.tolist()[0]
            stop = time.time()
            if epoch % self.show_epoch == 0:
                print("-"*50)
                print("epoch:", epoch)
                print("    loss:", loss)
                print("    target:%s\n    output:%s" % (tar, pre))
                print("    per-time:", (stop-start))
                torch.save(self.state_dict(), self.model_path+'params.pkl')

    def step(self, input_variable, target_variable, max_length):
        teacher_forcing_ratio = 0.1
        clip = 5.0
        loss = 0 # Added onto for each word

        self.encoder_optimizer.zero_grad()
        self.decoder_optimizer.zero_grad()

        input_length = input_variable.size()[0]
        target_length = target_variable.size()[0]

        encoder_hidden = self.encoder.init_hidden()
        encoder_outputs, encoder_hidden = self.encoder(input_variable, encoder_hidden)

        decoder_input = Variable(torch.LongTensor([[SOS_token]]))
        decoder_context = Variable(torch.zeros(1, self.decoder.hidden_size))
        decoder_hidden = encoder_hidden # Use last hidden state from encoder to start decoder
        if USE_CUDA:
            decoder_input = decoder_input.cuda()
            decoder_context = decoder_context.cuda()

        decoder_outputs = []
        use_teacher_forcing = random.random() < teacher_forcing_ratio
        use_teacher_forcing = True
        if use_teacher_forcing:
            for di in range(target_length):
                decoder_output, decoder_context, decoder_hidden, decoder_attention = self.decoder(decoder_input, decoder_context, decoder_hidden, encoder_outputs)
                loss += self.criterion(decoder_output, target_variable[di])
                decoder_input = target_variable[di]
                decoder_outputs.append(decoder_output.unsqueeze(0))
        else:
            for di in range(target_length):
                decoder_output, decoder_context, decoder_hidden, decoder_attention = self.decoder(decoder_input, decoder_context, decoder_hidden, encoder_outputs)
                loss += self.criterion(decoder_output, target_variable[di])
                decoder_outputs.append(decoder_output.unsqueeze(0))
                topv, topi = decoder_output.data.topk(1)
                ni = topi[0][0]

                decoder_input = Variable(torch.LongTensor([[ni]]))
                if USE_CUDA: decoder_input = decoder_input.cuda()
                if ni == EOS_token: break
        loss.backward()
        torch.nn.utils.clip_grad_norm(self.encoder.parameters(), clip)
        torch.nn.utils.clip_grad_norm(self.decoder.parameters(), clip)
        self.encoder_optimizer.step()
        self.decoder_optimizer.step()
        decoder_outputs = torch.cat(decoder_outputs, 0)
        return loss.data[0] / target_length, decoder_outputs

    def make_infer_fd(self, input_vec):
        inputs = []
        enc = input_vec[:self.max_length] if len(input_vec) > self.max_length else input_vec
        inputs.append(enc)
        inputs = Variable(torch.LongTensor(inputs)).transpose(1, 0).contiguous()
        if USE_CUDA:
            inputs = inputs.cuda()
        return inputs

    def predict(self):
        try:
            self.load_state_dict(torch.load(self.model_path+'params.pkl'))
        except Exception as e:
            print(e)
            print("No model!")
        loss_track = []

        # 加载字典
        str_to_vec = {}
        with open("./data/enc.vocab") as enc_vocab:
            for index,word in enumerate(enc_vocab.readlines()):
                str_to_vec[word.strip()] = index

        vec_to_str = {}
        with open("./data/dec.vocab") as dec_vocab:
            for index,word in enumerate(dec_vocab.readlines()):
                vec_to_str[index] = word.strip()

        while True:
            input_strs = input("me > ")
            # 字符串转向量
            segement = jieba.lcut(input_strs)
            input_vec = [str_to_vec.get(i, 3) for i in segement]
            input_vec = self.make_infer_fd(input_vec)

            # inference
            if self.beam_search:
                samples = self.beamSearchDecoder(input_vec)
                for sample in samples:
                    outstrs = []
                    for i in sample[0]:
                        if i == 1:
                            break
                        outstrs.append(vec_to_str.get(i, "Un"))
                    print("ai > ", "".join(outstrs), sample[3])
            else:
                logits = self.infer(input_vec)
                _,v = torch.topk(logits, 1)
                pre = v.cpu().data.numpy().T.tolist()[0][0]
                outstrs = []
                for i in pre:
                    if i == 1:
                        break
                    outstrs.append(vec_to_str.get(i, "Un"))
                print("ai > ", "".join(outstrs))

    def infer(self, input_variable):
        input_length = input_variable.size()[0]

        encoder_hidden = self.encoder.init_hidden()
        encoder_outputs, encoder_hidden = self.encoder(input_variable, encoder_hidden)

        decoder_input = Variable(torch.LongTensor([[SOS_token]]))
        decoder_context = Variable(torch.zeros(1, self.decoder.hidden_size))
        decoder_hidden = encoder_hidden
        if USE_CUDA:
            decoder_input = decoder_input.cuda()
            decoder_context = decoder_context.cuda()
        decoder_outputs = []

        for i in range(self.max_length):
            decoder_output, decoder_context, decoder_hidden, decoder_attention = self.decoder(decoder_input, decoder_context, decoder_hidden, encoder_outputs)
            decoder_outputs.append(decoder_output.unsqueeze(0))
            topv, topi = decoder_output.data.topk(1)
            ni = topi[0][0]
            decoder_input = Variable(torch.LongTensor([[ni]])) # Chosen word is next input
            if USE_CUDA: decoder_input = decoder_input.cuda()
            if ni == EOS_token: break

        decoder_outputs = torch.cat(decoder_outputs, 0)
        return decoder_outputs

    def tensorToList(self, tensor):
        return tensor.cpu().data.numpy().tolist()[0]

    def beamSearchDecoder(self, input_variable):
        input_length = input_variable.size()[0]
        encoder_hidden = self.encoder.init_hidden()
        encoder_outputs, encoder_hidden = self.encoder(input_variable, encoder_hidden)

        decoder_input = Variable(torch.LongTensor([[SOS_token]]))
        decoder_context = Variable(torch.zeros(1, self.decoder.hidden_size))
        decoder_hidden = encoder_hidden
        if USE_CUDA:
            decoder_input = decoder_input.cuda()
            decoder_context = decoder_context.cuda()

        decoder_output, decoder_context, decoder_hidden, decoder_attention = self.decoder(decoder_input, decoder_context, decoder_hidden, encoder_outputs)
        topk = decoder_output.data.topk(self.top_k)
        samples = [[] for i in range(self.top_k)]
        dead_k = 0
        final_samples = []
        for index in range(self.top_k):
            topk_prob = topk[0][0][index]
            topk_index = int(topk[1][0][index])
            samples[index] = [[topk_index], topk_prob, 0, 0, decoder_context, decoder_hidden, decoder_attention, encoder_outputs]

        for _ in range(self.max_length):
            tmp = []
            for index in range(len(samples)):
                tmp.extend(self.beamSearchInfer(samples[index], index))
            samples = []

            # 筛选出topk
            df = pd.DataFrame(tmp)
            df.columns = ['sequence', 'pre_socres', 'fin_scores', "ave_scores", "decoder_context", "decoder_hidden", "decoder_attention", "encoder_outputs"]
            sequence_len = df.sequence.apply(lambda x:len(x))
            df['ave_scores'] = df['fin_scores'] / sequence_len
            df = df.sort_values('ave_scores', ascending=False).reset_index().drop(['index'], axis=1)
            df = df[:(self.top_k-dead_k)]
            for index in range(len(df)):
                group = df.ix[index]
                if group.tolist()[0][-1] == 1:
                    final_samples.append(group.tolist())
                    df = df.drop([index], axis=0)
                    dead_k += 1
                    print("drop {}, {}".format(group.tolist()[0], dead_k))
            samples = df.values.tolist()
            if len(samples) == 0:
                break

        if len(final_samples) < self.top_k:
            final_samples.extend(samples[:(self.top_k-dead_k)])
        return final_samples

    def beamSearchInfer(self, sample, k):
        samples = []
        decoder_input = Variable(torch.LongTensor([[sample[0][-1]]]))
        if USE_CUDA:
            decoder_input = decoder_input.cuda()
        sequence, pre_scores, fin_scores, ave_scores, decoder_context, decoder_hidden, decoder_attention, encoder_outputs = sample
        decoder_output, decoder_context, decoder_hidden, decoder_attention = self.decoder(decoder_input, decoder_context, decoder_hidden, encoder_outputs)

        # choose topk
        topk = decoder_output.data.topk(self.top_k)
        for k in range(self.top_k):
            topk_prob = topk[0][0][k]
            topk_index = int(topk[1][0][k])
            pre_scores += topk_prob
            fin_scores = pre_scores - (k - 1 ) * self.alpha
            samples.append([sequence+[topk_index], pre_scores, fin_scores, ave_scores, decoder_context, decoder_hidden, decoder_attention, encoder_outputs])
        return samples

    def retrain(self):
        try:
            os.remove(self.model_path)
        except Exception as e:
            pass
        self.train()

if __name__ == '__main__':
    seq = seq2seq()
    if sys.argv[1] == 'train':
        seq.train()
    elif sys.argv[1] == 'predict':
        seq.predict()
    elif sys.argv[1] == 'retrain':
        seq.retrain()

### 2.3 Encoder Replacement with Convolutional or Self-attention-based Encoder

In [ ]:
# modified from https://github.com/yanwii/seq2seq/blob/master/seq2seq.py

# ENCODER

class EncoderCNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        
        super(EncoderCNN, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        
        self.CNN = nn.CNN()### TODO

    def forward(self, word_inputs, hidden):
        seq_len = len(word_inputs)
        embedded = self.embedding(word_inputs).view(seq_len, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def init_hidden(self):
        hidden = Variable(torch.zeros(self.n_layers, 1, self.hidden_size))
        return hidden

### 2.4 Fully self-attention Translation System

### 2.5 Multilingual Translation System